# Part 2 - Translation

You will modify Part 1 to generate the translations of your answers from Part 1 into a particular language (see below) and then back to English.

So, your prompt should look like:

> Your question.  
> Answer in English.  
> Answer in the assigned Language.  
> Answer in English, translated from the above language.

The language you will use for your project is:

Team 1, 4, 7, 10, 13 - Spanish

Team 2,5,8,11 - German

Team 3,6,9,12 - French

Observe the effects of the cyclical translation (e.g., English->French->English) and critique the results in your slides and the report.

Part 2.2 -- use two different HF translation models: use the default translation pipeline, then use other models of choice and discuss the differences in the result.

https://huggingface.co/docs/transformers/main_classes/pipelines

https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/pipelines#transformers.TranslationPipeline


In [1]:
!pip3 install -r ../requirements.txt

%load_ext autoreload
%autoreload 2


     ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/587.7 MB 15.0 MB/s eta 0:00:36^C
     ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/587.7 MB 17.0 MB/s eta 0:00:32
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from src.utils import read_context, read_questions
from src.translation import run, run_models


2023-11-14 16:04:58.145168: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


---
---

## Experiments & Results

You will modify Part 1 to generate the translations of your answers from Part 1 into a particular language (see below) and then back to English.

Observe the effects of the cyclical translation (e.g., English->French->English) and critique the results in your slides and the report.


In [ ]:
# TODO: Try out a good selection of models and keep some interesting ones
models = [
    "t5-base",
    "t5-large",
    "Helsinki-NLP/opus-mt-en-fr",
]


In [ ]:
## TODO: add experiments and results

test_txt = """
Sherlock Holmes took his bottle from the corner of the mantel-piece and
his hypodermic syringe from its neat morocco case. With his long,
white, nervous fingers he adjusted the delicate needle, and rolled back
his left shirt-cuff. For some little time his eyes rested thoughtfully
upon the sinewy forearm and wrist all dotted and scarred with
innumerable puncture-marks. Finally he thrust the sharp point home,
pressed down the tiny piston, and sank back into the velvet-lined
arm-chair with a long sigh of satisfaction.
"""


run_models(test_txt, models=models)


model: t5-base
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/home/acozma/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


> Sherlock Holmes took his bottle from the corner of the mantel-piece and
his hypodermic syringe from its neat morocco case. With his long,
white, nervous fingers he adjusted the delicate needle, and rolled back
his left shirt-cuff. For some little time his eyes rested thoughtfully
upon the sinewy forearm and wrist all dotted and scarred with
innumerable puncture-marks. Finally he thrust the sharp point home,
pressed down the tiny piston, and sank back into the velvet-lined
arm-chair with a long sigh of satisfaction.
> Sherlock Holmes a pris sa bouteille du coin du manteau et sa seringue hypodermique de son étui morocco. Avec ses longs doigts blancs et nerveux, il ajusta la délicate aiguille et roulait à l'envers de sa manche gauche. Pendant un peu de temps, ses yeux reposèrent soigneusement sur l'avant-bras
model: t5-large
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


/home/acozma/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


> Sherlock Holmes took his bottle from the corner of the mantel-piece and
his hypodermic syringe from its neat morocco case. With his long,
white, nervous fingers he adjusted the delicate needle, and rolled back
his left shirt-cuff. For some little time his eyes rested thoughtfully
upon the sinewy forearm and wrist all dotted and scarred with
innumerable puncture-marks. Finally he thrust the sharp point home,
pressed down the tiny piston, and sank back into the velvet-lined
arm-chair with a long sigh of satisfaction.
> Sherlock Holmes prenait sa bouteille de l'angle du manteau et sa seringue hypodermique de son étui morocain soigné. Avec ses longs doigts blancs et nerveux, il ajustait l'aiguille délicate et roulait son manteau gauche. Pendant un peu de temps, ses yeux restaient soigneusement sur l'avant-bra
model: Helsinki-NLP/opus-mt-en-fr
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
> Sherlock Holmes took his bottle from the corner of the mantel-pi